# **Layout Parser Model Training**

# Detectron2

Detectron2 is Facebook AI Research's next generation software system that implements state-of-the-art object detection algorithms. It is a ground-up rewrite of the previous version, Detectron, and it originates from maskrcnn-benchmark

### Installation
* detectron2 is not pre-installed in this kaggle docker, so let's install it.
* we need to know CUDA and pytorch version to install correct detectron2.

In [1]:
!nvidia-smi

Mon Feb  6 14:30:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.13.1+cpu False


* It seems CUDA=11.0 and torch==1.7.0 is used in this kaggle docker image.
* See installation for details. https://detectron2.readthedocs.io/en/latest/tutorials/install.html

### Install Pre-Built Detectron2

In [ ]:
!pip install "git+https://github.com/facebookresearch/detectron2.git"

# importing libraries

In [9]:
!conda install numba 

^C


In [2]:
import pandas as pd
import numpy as np
import pandas as pd 
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm # progress bar
from datetime import datetime
import time
import matplotlib.pyplot as plt
#from pycocotools.coco import COCO
import os, json, cv2, random
import skimage.io as io
import copy
from pathlib import Path
from typing import Optional


# torch
import torch

# Albumenatations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

#from pycocotools.coco import COCO
from sklearn.model_selection import StratifiedKFold

# glob
from glob import glob


import warnings
warnings.filterwarnings('ignore') #Ignore "future" warnings and Data-Frame-Slicing warnings.


# detectron2
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils


from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

setup_logger()

<Logger detectron2 (DEBUG)>

In [3]:
f = open(r'data-exported\COCO\result.json',)
thing_classes  = []
thing_classes_id = {}
data_annotations=[]
data = json.load(f)
#----Images----
data_images=data['images']
#---annotations-
for i in data['annotations']:
    annot_obj ={"id": i['id'],"image_id": i['image_id'],"category_id":i['category_id'],
          "x_min":i['bbox'][0], #left
          "y_min":i['bbox'][1], #top
          "x_max":i['bbox'][0]+i['bbox'][2], #left+width
          "y_max":i['bbox'][1]+i['bbox'][3] #top+hieght
         }
    data_annotations.append(annot_obj) 
#---categories-
for i in data['categories']:
    thing_classes.append(i['name'])
    thing_classes_id[i['name']]=i['id']
f.close()
print("thing_classes=",thing_classes)
print("thing_classes_id=",thing_classes_id)

thing_classes= ['DOB', 'EDate', 'FName', 'IDate', 'Id', 'LName', 'MName']
thing_classes_id= {'DOB': 0, 'EDate': 1, 'FName': 2, 'IDate': 3, 'Id': 4, 'LName': 5, 'MName': 6}


In [4]:
train_meta = pd.DataFrame(data_images)
train_meta = train_meta[['id', 'file_name', 'width', 'height']]
train_meta = train_meta.rename(columns={"id":"image_id"})
print("train_meta size=",len(train_meta))
train_meta.head(25)

train_meta size= 21


,image_id,file_name,width,height
0,0,77f17f42-IMG_6637.JPG,3024,4032
1,1,3c355f99-IMG_6638.JPG,3024,4032
2,2,3a9e546a-IMG_6639.JPG,3024,4032
3,3,0ce52bb6-IMG_6640.JPG,3024,4032
4,4,b91bed45-IMG_6642.JPG,3024,4032
5,5,2a36462b-IMG_6643.JPG,3024,4032
6,6,80b2ed21-IMG_6644.JPG,3024,4032
7,7,f9dfa2e6-IMG_6645.JPG,3024,4032
8,8,12fb4510-IMG_6646.JPG,3024,4032
9,9,48b45ec6-IMG_6647.JPG,3024,4032


In [5]:
train_df = pd.DataFrame(data_annotations)
print("train_df size=",len(train_df))
train_df.head(3)

train_df size= 145


,id,image_id,category_id,x_min,y_min,x_max,y_max
0,0,0,2,1174.017393,2097.193726,1449.882421,2152.132411
1,1,0,5,1804.118195,2076.787929,2117.601181,2139.574997
2,2,0,6,1454.584666,2086.205989,1793.146290,2142.714350


# configs

In [6]:
imgdir=r"data-exported\COCO\images"

debug=False
split_mode="valid20" # all_train Or  valid20 
image_Width=601
image_Height=792

num_folds=5
Selected_fold=1 #1,2,3,4,5 

In [7]:
def get_PL_data_dicts(
    imgdir: Path,
    _train_df: pd.DataFrame,
    _train_meta: pd.DataFrame,
    use_cache: bool = True,
    target_indices: Optional[np.ndarray] = None,
    debug: bool = False,
    data_type:str="train"
   
):
    if debug:
            train_meta = train_meta.iloc[:100]  # For debug...
    dataset_dicts = []
    for index, train_meta_row in tqdm(_train_meta.iterrows(), total=len(_train_meta)):
                    record = {}
                    image_id,file_name, width,height = train_meta_row.values
                    filename = str(f'{imgdir}/{file_name}')
                    record["file_name"] = filename
                    record["image_id"] = image_id
                    record["width"] = width
                    record["height"] = height
                    objs = []
                    for index2, row in _train_df.query("image_id == @image_id").iterrows():
                        class_id = row["category_id"]
                        bbox_resized = [
                            float(row["x_min"]),
                            float(row["y_min"]),
                            float(row["x_max"]),
                            float(row["y_max"]),
                        ]
                        obj = {
                            "bbox": bbox_resized,
                            "bbox_mode": BoxMode.XYXY_ABS,
                            "category_id": class_id,
                        }
                        objs.append(obj)
                    record["annotations"] = objs
                    dataset_dicts.append(record)
                    
    if target_indices is not None:
        dataset_dicts = [dataset_dicts[i] for i in target_indices]

    return dataset_dicts

                                  

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [9]:
Data_Resister_training="PL_data_train";
Data_Resister_valid="PL_data_valid";

if split_mode == "all_train":
    DatasetCatalog.register(
        Data_Resister_training,
        lambda: get_PL_data_dicts(
            imgdir,
            train_df,
            train_meta,
            debug=debug,
            data_type="train"
        ),
    )
    MetadataCatalog.get(Data_Resister_training).set(thing_classes=thing_classes)
    
    
    dataset_dicts_train = DatasetCatalog.get(Data_Resister_training)
    metadata_dicts_train = MetadataCatalog.get(Data_Resister_training)
    
    
elif split_mode == "valid20":

    n_dataset = len(train_meta)
    n_train = int(n_dataset * 0.95)
    print("n_dataset", n_dataset, "n_train", n_train)
    rs = np.random.RandomState(12)
    inds = rs.permutation(n_dataset)
    train_inds, valid_inds = inds[:n_train], inds[n_train:]
    DatasetCatalog.register(
        Data_Resister_training,
        lambda: get_PL_data_dicts(
            imgdir,
            train_df,
            train_meta,
            target_indices=train_inds,
            debug=debug,
            data_type="train"
        ),
    )
    MetadataCatalog.get(Data_Resister_training).set(thing_classes=thing_classes)
    

    DatasetCatalog.register(
        Data_Resister_valid,
        lambda: get_PL_data_dicts(
            imgdir,
            train_df,
            train_meta,
            target_indices=valid_inds,
            debug=debug,
            data_type="val"
            ),
        )
    MetadataCatalog.get(Data_Resister_valid).set(thing_classes=thing_classes)
    
    dataset_dicts_train = DatasetCatalog.get(Data_Resister_training)
    metadata_dicts_train = MetadataCatalog.get(Data_Resister_training)

    dataset_dicts_valid = DatasetCatalog.get(Data_Resister_valid)
    metadata_dicts_valid = MetadataCatalog.get(Data_Resister_valid)
    
else:
    raise ValueError(f"[ERROR] Unexpected value split_mode={split_mode}")

n_dataset 21 n_train 19


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [10]:
print("dicts training size=",len(dataset_dicts_train),"################  dicts valid size=",len(dataset_dicts_valid))

dicts training size= 19 ################  dicts valid size= 2


In [11]:
dataset_dicts_train[0]

{'file_name': 'data-exported\\COCO\\images/f9dfa2e6-IMG_6645.JPG',
 'image_id': 7,
 'width': 3024,
 'height': 4032,
 'annotations': [{'bbox': [1200.28955658144,
    1984.9294304464079,
    1524.958043197731,
    2088.382107081718],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 2.0},
  {'bbox': [2287.4370647965975,
    1960.297840771334,
    2744.924477755917,
    2073.6031532766738],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 6.0},
  {'bbox': [1524.9580431977288,
    1975.076794576377,
    2272.679406314035,
    2068.676835341656],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 4.0},
  {'bbox': [1210.1279955698105,
    2073.603153276671,
    2405.4983326570627,
    2211.540055457088],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0.0},
  {'bbox': [1111.743605686086,
    2374.1085473125736,
    1800.4343348721573,
    2477.5612239478855],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0.0},
  {'bbox': [1146.1781421453916,
    2649.

In [12]:
dataset_dicts_valid[0]

{'file_name': 'data-exported\\COCO\\images/80b2ed21-IMG_6644.JPG',
 'image_id': 6,
 'width': 3024,
 'height': 4032,
 'annotations': [{'bbox': [1101.367583801008,
    1926.5513021818324,
    1390.3183907814162,
    1990.4813894472386],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 2.0},
  {'bbox': [1806.945135729912,
    1913.0923364417467,
    2112.695408232437,
    1977.0224237071527],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 6.0},
  {'bbox': [1397.0381769902629,
    1926.5513021818324,
    1813.6649219387587,
    1983.7519065771958],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 5.0},
  {'bbox': [1101.3675838010079,
    1987.1166480122165,
    2005.1788288908892,
    2084.694149627837],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0.0},
  {'bbox': [1111.4472631142783,
    2215.9190655936695,
    1575.112511524701,
    2289.9433771641397],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 3.0},
  {'bbox': [1108.0873700098548,
  

# Data Visualization

fig = plt.figure(figsize =(40,40))
ax = fig.add_subplot(1,1,1)

d=dataset_dicts_valid[1]   
img = cv2.imread(d["file_name"])
v = Visualizer(img[:, :, ::-1],
                metadata=metadata_dicts_valid, 
                scale=1.5, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
out = v.draw_dataset_dict(d)
ax.grid(False)
ax.axis('off')
ax.imshow(out.get_image()[:, :, ::-1])

In [ ]:
fig = plt.figure(figsize =(40,40)) 
ax = fig.add_subplot(1,1,1)

d=dataset_dicts_valid[1]
img = cv2.imread(d["file_name"]) 
v = Visualizer(img[:, :, ::-1], metadata=metadata_dicts_valid, scale=1.5, instance_mode=ColorMode.IMAGE_BW) # remove the colors of unsegmented pixels. This option is only available for segmentation models ) 
out = v.draw_dataset_dict(d) 
ax.grid(False) 
ax.axis('off') 
ax.imshow(out.get_image()[:, :, ::-1])

# Data Augmentation

In [13]:
def custom_mapper(dataset_dict):
    
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [T.RandomBrightness(0.8, 1.2),
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False)
                      ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict
class AugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

In [16]:
cfg = get_cfg()
config_name = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml" 

cfg.merge_from_file(model_zoo.get_config_file(config_name))

cfg.DATASETS.TRAIN = (Data_Resister_training,)

if split_mode == "all_train":
    cfg.DATASETS.TEST = ()
else:
    cfg.DATASETS.TEST = (Data_Resister_valid,)
    cfg.TEST.EVAL_PERIOD = 1000

cfg.DATALOADER.NUM_WORKERS = 0
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_name)
cfg.MODEL.WEIGHTS="kaggle/Weights/mask_rcnn_R_50_FPN_3x/model_final.pth"

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0025

cfg.SOLVER.WARMUP_ITERS = 10
cfg.SOLVER.MAX_ITER = 2000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (500, 1000) # must be less than  MAX_ITER 
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.DEVICE = "cpu"


cfg.SOLVER.CHECKPOINT_PERIOD = 100000  # Small value=Frequent save need a lot of storage.
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7 


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


#Training using custom trainer defined above
trainer = AugTrainer(cfg) 
#trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[02/07 05:08:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

  0%|          | 0/21 [00:00<?, ?it/s]

[02/07 05:08:26 d2.data.build]: Removed 0 images with no usable annotations. 19 images left.
[02/07 05:08:26 d2.data.build]: Using training sampler TrainingSampler
[02/07 05:08:26 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/07 05:08:26 d2.data.common]: Serializing 19 elements to byte tensors and concatenating them all ...
[02/07 05:08:26 d2.data.common]: Serialized dataset takes 0.01 MiB
[02/07 05:08:26 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from kaggle/Weights/mask_rcnn_R_50_FPN_3x/model_final.pth ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (7, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (7,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (24, 1024) in the checkpoint but (28, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (24,) in the checkpoint but (28,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weig

[02/07 05:08:26 d2.engine.train_loop]: Starting training from iteration 0
ERROR [02/07 05:08:45 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "d:\miniconda3\envs\cv2\lib\site-packages\detectron2\engine\train_loop.py", line 149, in train
    self.run_step()
  File "d:\miniconda3\envs\cv2\lib\site-packages\detectron2\engine\defaults.py", line 494, in run_step
    self._trainer.run_step()
  File "d:\miniconda3\envs\cv2\lib\site-packages\detectron2\engine\train_loop.py", line 274, in run_step
    loss_dict = self.model(data)
  File "d:\miniconda3\envs\cv2\lib\site-packages\torch\nn\modules\module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "d:\miniconda3\envs\cv2\lib\site-packages\detectron2\modeling\meta_arch\rcnn.py", line 158, in forward
    features = self.backbone(images.tensor)
  File "d:\miniconda3\envs\cv2\lib\site-packages\torch\nn\modules\module.py", line 1194, in _call_impl
    return forward_call(*

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3137863680 bytes.

In [ ]:
evaluator = COCOEvaluator(Data_Resister_training, cfg, False, output_dir="./output/")
cfg.MODEL.WEIGHTS="./output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
val_loader = build_detection_test_loader(cfg, Data_Resister_training)
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
import pandas as pd
metrics_df = pd.read_json("./output/metrics.json", orient="records", lines=True)
mdf = metrics_df.sort_values("iteration")
mdf.head(10).T

In [ ]:
fig, ax = plt.subplots()

mdf1 = mdf[~mdf["total_loss"].isna()]
ax.plot(mdf1["iteration"], mdf1["total_loss"], c="C0", label="train")
if "validation_loss" in mdf.columns:
    mdf2 = mdf[~mdf["validation_loss"].isna()]
    ax.plot(mdf2["iteration"], mdf2["validation_loss"], c="C1", label="validation")

# ax.set_ylim([0, 0.5])
ax.legend()
ax.set_title("Loss curve")
plt.show()

In [ ]:
fig, ax = plt.subplots()

mdf1 = mdf[~mdf["fast_rcnn/cls_accuracy"].isna()]
ax.plot(mdf1["iteration"], mdf1["fast_rcnn/cls_accuracy"], c="C0", label="train")
# ax.set_ylim([0, 0.5])
ax.legend()
ax.set_title("Accuracy curve")
plt.show()

In [ ]:
fig, ax = plt.subplots()
mdf1 = mdf[~mdf["loss_box_reg"].isna()]
ax.plot(mdf1["iteration"], mdf1["loss_box_reg"], c="C0", label="train")
# ax.set_ylim([0, 0.5])
ax.legend()
ax.set_title("loss_box_reg")
plt.show()

In [ ]:
!pip install -U layoutparser 

In [ ]:
import layoutparser as lp

In [ ]:
model = lp.Detectron2LayoutModel('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                                 './output/model_final.pth',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                 label_map={0:"None",1: "text", 2: "title", 3: "list", 4:"table", 5:"figure"})
#######################################################
image = cv2.imread('/kaggle/input/papers-images/train/train/PMC3576793_00004.jpg')
#plt.imshow(image)
color_map = {
    'text':   'red',
    'title':  'blue',
    'list':   'green',
    'table':  'yellow',
    'figure': 'pink',
}

layout_predicted = model.detect(image)
lp.draw_box(image,
              [b.set(id=f'{b.type}/{b.score:.2f}') for b in layout_predicted],
              color_map=color_map,
              show_element_id=True, id_font_size=10,
              id_text_background_color='black',
              id_text_color='white')

In [ ]:
cfg.MODEL.WEIGHTS = "./output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
#cfg.DATASETS.TEST = ("Data_Resister_training", )
predictor = DefaultPredictor(cfg)

###############################################################

fig, ax = plt.subplots(2, 2, figsize =(20,20))
indices=[ax[0][0],ax[1][0],ax[0][1],ax[1][1] ]
i=-1
# Show some qualitative results by predicting on test set images
NUM_TEST_SAMPLES = 4
samples = random.sample(dataset_dicts_train, NUM_TEST_SAMPLES)
for i, sample in enumerate(samples):
    img = cv2.imread(sample["file_name"])
    outputs = predictor(img)
    visualizer = Visualizer(img, metadata=metadata_dicts_train,scale=0.5,)
    visualizer = visualizer.draw_instance_predictions(
        outputs["instances"].to("cpu"))
    display_img = visualizer.get_image()[:, :, ::-1]
    indices[i].grid(False)
    indices[i].imshow(display_img)

### References
* https://layout-parser.readthedocs.io/en/latest/notes/installation.html
* https://github.com/Layout-Parser/layout-model-training/blob/master/tools/train_net.py
* https://towardsdatascience.com/auto-parse-and-understand-any-document-5d72e81b0be9
* https://layout-parser.readthedocs.io/en/latest/api_doc/models.html